In [0]:
%sql
CREATE TABLE IF NOT EXISTS odag.gold.ODAG_FINAL_TABLE
(
SOURCE_NAME VARCHAR(25),
REQUEST_ID INT,
FIRST_NAME VARCHAR(50),
LAST_NAME VARCHAR(50),
CONTRACT_ID VARCHAR(5),
DATE_FORWARDED_TO_IRE VARCHAR(10),
CHG_TIMESTAMP STRING,
REQID_UNIV_SUBPART_KEY VARCHAR(40)
)


In [0]:
request_id = spark.sql("SELECT Request_ID FROM global_temp.status_tracker_gtv")
request_id_value = request_id.collect()[0]['Request_ID']

In [0]:
%sql truncate table odag.gold.ODAG_FINAL_TABLE

In [0]:
%python
sql_query = spark.sql(f"""
INSERT INTO odag.gold.ODAG_FINAL_TABLE
SELECT 
    CASE WHEN SOURCE_INDICATOR IS NULL OR trim(SOURCE_INDICATOR) = '' 
            THEN 'NA' 
        ELSE 
            SUBSTR(SOURCE_INDICATOR,1,25) 
    END AS `SOURCE_NAME`,
    {request_id_value} AS `REQUEST_ID`,
    CASE WHEN FIRST_NAME IS NULL OR trim(FIRST_NAME) = '' 
            THEN 'NA' 
        ELSE 
            SUBSTR(FIRST_NAME,1,50) 
    END AS `FIRST_NAME`,
    CASE WHEN LAST_NAME IS NULL OR trim(LAST_NAME) = '' 
            THEN 'NA' 
        ELSE 
            SUBSTR(LAST_NAME,1,50) 
    END AS `LAST_NAME`,
    CASE WHEN CONTRACT_ID IS NULL OR trim(CONTRACT_ID) = '' 
            THEN 'NA' 
        ELSE 
            SUBSTR(CONTRACT_ID,1,5) 
    END AS `CONTRACT_ID`,
    CASE WHEN DATE_FORWARDED_TO_IRE IS NULL OR trim(DATE_FORWARDED_TO_IRE) = '' 
            THEN 'NA' 
        ELSE 
            SUBSTR(DATE_FORWARDED_TO_IRE,1,10) 
    END AS `DATE_FORWARDED_TO_IRE`,
    date_format(current_timestamp(), 'MM/dd/yyyy h:mm:ss a') AS `CHG_TIMESTAMP`,
    SUBSTR(CONCAT('{request_id_value}', '_M_', date_format(current_timestamp(), 'yyyyMMddHHmmss')),1,40) AS `REQID_UNIV_SUBPART_KEY`
FROM odag.bronze.CMS_AUDIT_INPUT_STG
""")

In [0]:
%sql
SELECT * FROM odag.gold.ODAG_FINAL_TABLE